In [1]:
import pandas as pd
import numpy as np
import sqlite3
import ast
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
user_response = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_responses.csv')

In [ ]:
user_response.rename(columns = {'What country are you from? ': 'nationality', 'Age Range': 'age', 'Gender': 'gender',
                    '1. Choose your top favorite 3-5 cities you have traveled to that are on this list.  - Favorite City #1': 'favorite_city_one',
                    '2. Favorite city #2': 'favorite_city_two', '3. Favorite city #3': 'favorite_city_three',
                    '4. Favorite city #4': 'favorite_city_four', '5. Favorite city #5': 'favorite_city_five',
                    "6. If there's a city you have been and loved that is not on this list, add it below. ": 'extra_favorite',
                    "7. What cities on this list have you been to and not enjoyed?   - Least favorite city #1": 'least_favorite_one',
                    '8. Least favorite city #2': 'least_favorite_two', 
                    "9. If there's a city you have been to and haven't liked that is not on this list, add it below": 'extra_least_favorite',
                    "What price range of restaurant do you eat at when you travel? [Price level: 1 (Fast/Cheap Eats)]": 'food_one',
                    "What price range of restaurant do you eat at when you travel? [Price level: 2 (Casual Dining)]": 'food_two',
                    "What price range of restaurant do you eat at when you travel? [Price level: 3 (Upscale Dining)]": 'food_three',
                    "What price range of restaurant do you eat at when you travel? [Price level: 4 (Fine Dining/High End)]": 'food_four',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Art Gallery]": 'art_gallery',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Library]": 'library',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Museum ]": 'museum',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Aquarium]": 'aquarium',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Amusement Park ]": 'amusement_park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Zoo]": 'zoo',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Movie Theater]": 'movie_theater',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Mall / Souvenir shop ]": 'shop',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Park ]": 'park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Natural Feature / Beach]": 'natural_feature',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Place of Worship (Church/Temple)]": 'place_of_worship'}, inplace=True )

In [ ]:
user_response.info()

In [ ]:
user_response.fillna('Zx', inplace=True)

In [ ]:
user_response.drop(columns=['Timestamp', 'extra_favorite', 'extra_least_favorite', 'least_favorite_one', 'least_favorite_two'], inplace=True)

In [ ]:
nationality_dict = {'Australia': 1, 'Canada': 2, 'China': 3, 'Finland': 4, 'Honduras': 5,
              'India': 6, 'Israel': 7, 'Japan': 8, 'Mexico': 9, 'Pakistan': 10, 'Philippines': 11, 'United States': 12}

user_response.nationality = user_response.nationality.map(nationality_dict)


In [ ]:
user_response

In [3]:
db = get_df('cities')
db = db.drop(db[db.id == '80'].index)

,id,city,country
79,80,Guilin,China


In [ ]:
new_row = {'id': 200, 'city': 'Zx', 'country': 'None'}

db = db.append(new_row, ignore_index=True)

In [ ]:
db[db['id'] == '80']

In [ ]:
# Remove user with favorite city = Guilin - no restuarants were found not enough data
user_response.loc[108, 'favorite_city_four'] = 'Zx'

In [ ]:
from sklearn import preprocessing as pp

le = pp.LabelEncoder()
le.fit(db.city)
#numpy.save('classes.npy', encoder.classes_)

user_response['one'] = le.transform(user_response['favorite_city_one'])
user_response

In [ ]:
user_response['two'] = le.transform(user_response['favorite_city_two'])
user_response['three'] = le.transform(user_response['favorite_city_three'])
user_response['four'] = le.transform(user_response['favorite_city_four'])
user_response['five'] = le.transform(user_response['favorite_city_five'])


In [ ]:
user_response.movie_theater.value_counts()

In [ ]:
slim_response = user_response.drop(columns=['favorite_city_one', 'favorite_city_two', 'favorite_city_three',
                                          'favorite_city_four', 'favorite_city_five'])

In [ ]:
slim_response

In [ ]:
replace_map = {'1 ( Would NOT go)': 1, '2': 2, '3': 3, '4 (Definitely would go)': 4}

slim_response = slim_response.replace(replace_map)

In [ ]:
replace_map_food = {'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often': 3, 'Always': 4}

slim_response = slim_response.replace(replace_map_food)


In [ ]:
slim_response

In [ ]:
dummy = pd.get_dummies(slim_response)


In [ ]:
dummy

In [ ]:
for c in user_response.columns:
    print(c)

In [ ]:
for c in dummy.columns:
    print(c)

In [ ]:
dummy


In [ ]:
cosine_user = cosine_similarity(dummy)


In [ ]:
# cosine does not have user demographics 
cosine_user = pd.DataFrame(cosine_user)
cosine_user

In [ ]:
top_10_user = find_similar_n(cosine_user,10)
top_10_user

## Building city matrix


In [ ]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')
all_food = pd.concat([one, two, three, four, top_rest], axis =0)
all_food.sample(5)

In [ ]:
all_food['price_level'] = all_food['price_level'].fillna(all_food.groupby('city')['price_level'].transform('mean'))
#Beirut and Tehran are still NA. fill na with 2?
all_food.fillna(2.0, inplace=True)

In [ ]:
all_food['price_level'] = all_food['price_level'].astype(int)


In [ ]:
attractions = get_df('attractions')
attractions.sample(5)

In [ ]:
attractions[attractions['city'] == 'Cancún']

In [ ]:
attractions.shape

In [ ]:
attractions.info()

In [ ]:
def to_city(df):
    city_df = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count().to_frame()
    price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
    price_level['avg_price'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()

    
    return price_level

In [ ]:
food = to_city(all_food)
food

In [ ]:
place_of_worship = ['place_of_worship', 'hindu_temple', 'church', 'mosque', 'synagogue']
shopping = ['store', 'shopping_mall', 'clothing_store', 'electronics_store', 'grocery_or_supermarket', 'department_store']

attractions_to_keep = ['amusement_park', 'museum', 'park', 'art_gallery', 'aquarium',
                      'zoo', 'library', 'movie_theater', 'natural_feature'] + place_of_worship + shopping

In [ ]:
def main_call(attraction_df):
    attractions_split = split_types(attraction_df)
    dummy = dummies(attractions_split)
    by_city, all_attractions = attraction_count(dummy, attractions_split)
    
    return by_city, all_attractions

def split_types(df):
    df['split_types'] = [ast.literal_eval(x) for x in df.types]
    df['split_types_str'] = [','.join(x) for x in df.split_types]
    
    return df

def dummies(df):
    dummies = df.split_types_str.str.get_dummies(sep=',')

    return dummies

def attraction_count(dummies_df, all_attractions_df):
    all_attractions_df = pd.concat([all_attractions_df, dummies_df], axis=1)
    type_col_names = attractions_to_keep
    type_col_names.extend(['country', 'city', 'id'])
    attraction_count = all_attractions_df[type_col_names].groupby(['country', 'city', 'id']).sum()
    
    return attraction_count, all_attractions_df

In [ ]:
city_group, all_attractions = main_call(attractions)

In [ ]:
city_group

#### Adding columns for worship and stores

In [ ]:
city_group['place_of_worship2'] = city_group['place_of_worship'] + city_group['hindu_temple'] + city_group['church'] + city_group['mosque'] + city_group['synagogue']
city_group['store2'] = city_group['store'] + city_group['shopping_mall'] + city_group['clothing_store'] + city_group['electronics_store'] + city_group['grocery_or_supermarket'] + city_group['department_store']


In [ ]:
city_group.drop(columns=['place_of_worship', 'hindu_temple', 'church', 'mosque', 'synagogue', 'department_store',
                        'grocery_or_supermarket', 'electronics_store', 'clothing_store', 'shopping_mall', 'store'], inplace=True)
city_group

In [ ]:
city_group.rename(columns={"place_of_worship2" : 'place_of_worship', 'store2': 'store'}, inplace=True)

In [ ]:
city_group.reset_index(inplace=True)

In [ ]:
city_group = city_group.drop(city_group[city_group.city == 'Guilin'].index)

In [ ]:
city_group.set_index(['country', 'city', 'id'], inplace=True)

In [ ]:
def attraction_food(attraction_df, food_df):
    city = pd.merge(attraction_df, food_df, on = ['country', 'city', 'id'], how = 'outer')
    city.reset_index(inplace=True)
    city['id'] = pd.to_numeric(city['id'])
    city.set_index(['country', 'city', 'id'], inplace=True)
    city.sort_values('id', inplace=True)
    return city

In [ ]:
food.fillna(0.0, inplace=True)

In [ ]:
city = attraction_food(food, city_group)

In [ ]:
city.info()

In [ ]:
city

In [ ]:
cosine_city = pd.DataFrame(cosine_similarity(city))

cosine_city

In [ ]:
top_10_city = find_similar_n(cosine_city, 10)
top_10_city

### All user and city matrix completed

In [ ]:
def select_favorite():
    for user in user_response.iloc[k]:
        one =  city.iloc[city.index.get_level_values('city') == user_response.iloc[k]['favorite_city_one']]
        two =  city.iloc[city.index.get_level_values('city') == user_response.iloc[k]['favorite_city_two']]
        three =  city.iloc[city.index.get_level_values('city') == user_response.iloc[k]['favorite_city_three']]
        top = pd.concat([one, two, three])
        top = top.stack().to_frame().reset_index(drop=True).T

In [ ]:
test = user_response.iloc[0:3].copy()
test

In [ ]:
user_response.head()

In [ ]:

def build_top_city(city_matrix, user_matrix):
    df = pd.DataFrame([])
    k = 0
    while k < len(user_matrix):
        one =  city_matrix.iloc[city_matrix.index.get_level_values('city') == user_matrix.iloc[k]['favorite_city_one']]
        one.reset_index(inplace=True)
        one = one.add_suffix('_1')
        two =  city_matrix.iloc[city_matrix.index.get_level_values('city') == user_matrix.iloc[k]['favorite_city_two']]
        two.reset_index(inplace=True)
        two = two.add_suffix('_2')
        three =  city_matrix.iloc[city_matrix.index.get_level_values('city') == user_matrix.iloc[k]['favorite_city_three']]
        three.reset_index(inplace=True)
        three = three.add_suffix('_3')
        top = pd.concat([one, two, three])
        top.drop(columns=['country_1', 'city_1', 'country_2', 'city_2', 'country_3', 'city_3'], inplace=True)
        top = top.stack().to_frame().T
        top.columns = top.columns.droplevel()
        df = df.append(top)
        df.reset_index(drop=True, inplace=True)

        k += 1
        
    return df
    
    
def build_design_matrix(dummy_response, top_cities):
    finished = pd.concat([dummy_response, top_cities], axis=1)
    return finished

def find_top_city(city_matrix, user_matrix):
    k = 0
    i = 3
    j = '1'
    df = pd.DataFrame([])
    for k in range(len(user_matrix)):
        while i < 6:
            one =  city_matrix.iloc[city_matrix.index.get_level_values('city') == user_matrix.iloc[k][i]]
            one.reset_index(inplace=True)  
            one = one.add_suffix('_'+j)
            i += 1
        
    return one


    
    


In [ ]:
top_3 = build_top_city(city, user_response)
top_3

In [ ]:
dummy

In [ ]:
matrix = build_design_matrix(dummy, top_3)
matrix

In [ ]:
matrix.iloc[:, 485:]

In [ ]:
matrix.isnull().any()

In [ ]:
matrix[matrix['shopping_mall_3'].isnull()]

In [ ]:
matrix.drop([9, 17, 25, 33, 40, 43, 45, 59, 82, 140], inplace=True)

In [ ]:
# Need to remove country/city in the top three matrix before appending.
cos = cosine_similarity(matrix)
cos = pd.DataFrame(cos)

In [ ]:
#This will create a df of any null values
nan_rows = matrix[matrix.isnull().T.any().T]

In [ ]:
nan_rows

In [ ]:
cos

In [ ]:
bla = find_similar_n(cos, 10)
bla

### Design matrix is complete.

Each row is a user, with the survey response and their top 3 countries info (food, price, attractions)

In [ ]:
matrix.sample(3)

In [ ]:
y = matrix.id_3

In [ ]:
full_df = matrix.copy()

In [ ]:
df = matrix.copy()

In [ ]:
dropthat = df.iloc[:, 487:].columns

In [ ]:
df.drop(columns=dropthat, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [ ]:
dropthat

In [ ]:
from scipy import linalg
U, s, Vh = linalg.svd(full_df)

In [ ]:
full_df

### KNN is not effective really low prediction score.

Best k = 1. 
Best k = 7? 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 4
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(df,y)
neigh

In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

In [ ]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

In [ ]:
y_train

In [ ]:
# write your code here
k2 = 6
new = KNeighborsClassifier(n_neighbors=k2)
new.fit(X_train, y_train)

yhat2 = neigh.predict(X_test)

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, new.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat2))

In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

### Creating userxcity matrix with dummy values

In [ ]:
user = dummy

In [ ]:
city_ = city.drop(columns=['avg_price'])

In [ ]:
user

In [ ]:
city_.reset_index(inplace=True)

In [ ]:
city_[city_['city'] == 'Cancun']

In [ ]:
city_ = city_.drop(city_[city_.city == 'Cancún'].index)

In [ ]:
city_['label_id'] = le.transform(city_['city'])



In [ ]:
user.reset_index(drop=False, inplace=True)


In [ ]:
city_['key'] = 1
user['key'] = 1



In [ ]:
user

In [ ]:
df3 = pd.merge(city_,user,on='key').drop('key',axis=1)


In [ ]:

df3

In [ ]:
user

In [ ]:
df3.drop(columns=['country', 'id'], inplace=True)

In [ ]:
df3.set_index(['city', 'label_id', 'index'], inplace=True)

In [ ]:
for c in df3.columns:
    print(c)

In [ ]:
df3.iloc[:, 16:25]

In [ ]:
df3.reset_index(level=1, drop=False, inplace=True)

In [ ]:
def add_rank(x):
    
    if x.one == x.label_id:
        return 1
    if x.two == x.label_id:
        return 1
    if x.three == x.label_id:
        return 1
    if x.four == x.label_id:
        return 1
    if x.five == x.label_id:
        return 1
    else:
        return 0
    
    
    

In [ ]:
df3['rank'] = df3.apply(add_rank,axis=1)


In [ ]:
df3.reset_index(drop=False, inplace=True)

df3.rename(columns={'index': 'user'}, inplace=True)

In [ ]:
df3

In [ ]:
df3.columns

In [ ]:
df3.sort_values(['user', 'city'], inplace=True)


In [ ]:
df3

In [ ]:
df3.set_index(['user', 'city', 'label_id'], inplace=True)

In [ ]:
df3

In [ ]:
df3.to_csv(r'/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv')

In [ ]:
test = df3[df3['rank'] ==1].copy()

In [ ]:
test.iloc[:, 16:]

In [ ]:
test